In [1]:
from typing import Tuple, Dict

In [2]:
demo_dataset = (4, 8)
full_dataset = (9, 4)
demo_dataset

(4, 8)

# Part 1

In [3]:
class DeterministicDice:
    
    def __init__(self):
        self.current = 0
        self.rolls = 0
        
    def next(self) -> int:
        self.current = 1 if self.current == 100 else (self.current + 1)
        self.rolls += 1
        return self.current
    
    def triple_next(self) -> int:
        return self.next() + self.next() + self.next()

In [4]:
def get_next_score(current_score: int, dice: DeterministicDice) -> int:
    new_score = current_score + dice.triple_next()
    new_score = new_score % 10
    return 10 if new_score == 0 else new_score

In [5]:
def get_final_score(positions: Tuple[int]) -> int:
    p1 = positions[0]
    p2 = positions[1]
    score1 = 0
    score2 = 0
    dice = DeterministicDice()
    while True:
        p1 = get_next_score(p1, dice)
        score1 += p1
        if score1 >= 1000:
            return score2 * dice.rolls
        p2 = get_next_score(p2, dice)
        score2 += p2
        if score2 >= 1000:
            return score1 * dice.rolls
        
    

In [6]:
assert get_final_score(demo_dataset) == 739785

In [7]:
get_final_score(full_dataset)

998088

# Part 2

In [8]:
def dirac_dice_outcomes() -> Dict[int, int]:
    scores = [0]
    for i in range(3):
        new_scores = []
        for score in scores:
            for dice in range(1, 4):
                new_scores.append(score + dice)
        scores = new_scores
    result = {}
    for score in scores:
        result[score] = result.get(score, 0) + 1
    return result
    
    
DIRAC_RESULTS = dirac_dice_outcomes()
DIRAC_RESULTS

{3: 1, 4: 3, 5: 6, 6: 7, 7: 6, 8: 3, 9: 1}

In [9]:
def increase_position(position: int, increase: int) -> int:
    new_position = (position + increase) % 10
    return 10 if new_position == 0 else new_position

In [10]:
from functools import lru_cache

@lru_cache(20000)
def get_number_win(p1: int, p2: int, score1: int , score2: int, turn: int) -> Tuple[int, int]:
    if score1 >= 21:
        return (1, 0)
    elif score2 >= 21:
        return (0, 1)
    else:
        win1, win2 = 0, 0
        for value, count in DIRAC_RESULTS.items():
            if turn == 0:
                new_p1 = increase_position(p1, value)
                new_scores = get_number_win(new_p1, p2, score1 + new_p1, score2, 1)
            else:
                new_p2 = increase_position(p2, value)
                new_scores = get_number_win(p1, new_p2, score1, score2 + new_p2, 0)    
            win1 += count * new_scores[0]
            win2 += count * new_scores[1]
        return (win1, win2)
        
        

In [11]:
def get_dirac_score(dataset) -> int:
    scores = get_number_win(dataset[0], dataset[1], 0, 0, 0)
    return max(scores)

In [12]:
assert get_dirac_score(demo_dataset) == 444356092776315

In [13]:
get_dirac_score(full_dataset)

306621346123766